## 1


In [8]:
import numpy as np
import os
import soundfile as sf
from scipy.signal import lfilter

# =====================================================
# 1) LISTAR OS WAV QUE JÁ ESTÃO NA PASTA
# =====================================================

folder = "FragileThoughts/"   # <-- altera se a tua pasta tiver outro nome

available_wavs = [f for f in os.listdir(folder) if f.lower().endswith(".wav")]

for w in available_wavs:
    print(" -", w)

# Selecionar 7 pistas (ou menos, se ainda não tens 7)
selected = available_wavs[:7]

for s in selected:
    print(" -", s)

# =====================================================
# 2) FUNÇÕES DO MODELO ACÚSTICO
# =====================================================

def apply_delay(signal, delay_samples):
    out = np.zeros(len(signal) + delay_samples)
    out[delay_samples:delay_samples+len(signal)] = signal
    return out

def schroeder_reverb(x, Fs, RT60):
    if RT60 == 0:
        return np.zeros_like(x)

    comb_delays = np.array([1557, 1617, 1491, 1422])
    ap_delays = np.array([225, 556])
    g_comb = 10 ** (-3 * comb_delays / (RT60 * Fs))

    comb_outputs = []
    for delay, g in zip(comb_delays, g_comb):
        b = np.zeros(delay+1)
        b[0] = 1
        b[-1] = -g
        a = np.zeros(delay+1)
        a[0] = 1
        comb_outputs.append(lfilter(b, a, x))

    y = sum(comb_outputs)

    g_ap = 0.7
    for delay in ap_delays:
        b = np.zeros(delay+1)
        a = np.zeros(delay+1)
        b[0] = -g_ap
        b[-1] = 1
        a[0] = 1
        a[-1] = g_ap
        y = lfilter(b, a, y)

    return y

def modelo_FIR_com_reverb(x, m, a, Fs, RT60):
    y1 = apply_delay(x, m[0]) * a[0]
    y2 = apply_delay(x, m[0] + m[1]) * a[1]
    y3 = apply_delay(x, m[0] + m[1] + m[2]) * a[2]

    xr = apply_delay(x, m[0] + m[1] + m[2])
    yR = schroeder_reverb(xr, Fs, RT60)

    L = max(len(y1), len(y2), len(y3), len(yR))
    y1 = np.pad(y1, (0, L-len(y1)))
    y2 = np.pad(y2, (0, L-len(y2)))
    y3 = np.pad(y3, (0, L-len(y3)))
    yR = np.pad(yR, (0, L-len(yR)))

    return y1 + y2 + y3 + yR

# =====================================================
# 3) PARÂMETROS ACÚSTICOS
# =====================================================

Fs = 48000
c = 343

d1 = 1.5
d2 = 2.2
d3 = 3.1

m1 = int((d1 / c) * Fs)
m2 = int((d2 / c) * Fs)
m3 = int((d3 / c) * Fs)
m = [m1, m2, m3]

a1 = 1/d1
a2 = 1/d2
a3 = 1/d3
a = [a1, a2, a3]

# =====================================================
# 4) CARREGAR UMA DAS PISTAS
# =====================================================

track_to_use = selected[0]   # Usa a primeira pista para a Questão 1
path = folder + track_to_use

x, Fs = sf.read(path)
print("\nA processar:", track_to_use)

# =====================================================
# 5) GERAR OS 4 SINAIS
# =====================================================

y_RT0   = modelo_FIR_com_reverb(x, m, a, Fs, RT60=0)
y_RT05  = modelo_FIR_com_reverb(x, m, a, Fs, RT60=0.5)
y_RT2   = modelo_FIR_com_reverb(x, m, a, Fs, RT60=2)
y_RT10  = modelo_FIR_com_reverb(x, m, a, Fs, RT60=10)

# =====================================================
# 6) GUARDAR OUTPUTS
# =====================================================

sf.write("campo_RT0.wav",  y_RT0,  Fs)
sf.write("campo_RT05.wav", y_RT05, Fs)
sf.write("campo_RT2.wav",  y_RT2,  Fs)
sf.write("campo_RT10.wav", y_RT10, Fs)

print("\nFicheiros gerados com sucesso!")


 - 01_Kick.wav
 - 02_Snare.wav
 - 03_Overhead.wav
 - 04_BassDI.wav
 - 05_Gtr.wav
 - 06_Percussion.wav
 - 07_LeadVox.wav
 - 01_Kick.wav
 - 02_Snare.wav
 - 03_Overhead.wav
 - 04_BassDI.wav
 - 05_Gtr.wav
 - 06_Percussion.wav
 - 07_LeadVox.wav

A processar: 01_Kick.wav

Ficheiros gerados com sucesso!
